In [ ]:
import tensorflow as tf
import torch

# check if GPU is being used
print("GPU Status: ", end = '')
if len(tf.config.list_physical_devices('GPU')) != 0: print("Active")
else: print("Inactive")

In [ ]:
# TensorFlow and tf.keras
import numpy as np
from tensorflow import keras
import autokeras as ak

#tf version should be 2.5 or higher
tf.__version__

In [ ]:
#get data
(train_images, train_labels), (test_images, test_labels) = \
      keras.datasets.fashion_mnist.load_data()

In [ ]:
#scale data
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
#reshape inputs for CNN layers
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [ ]:
import os
from tensorboard.plugins.hparams import api as hp

# set hparams
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'adagrad', 'rmsprop', 'sgd']))
HP_CONV_UNITS = hp.HParam('conv_units', hp.Discrete([6, 12, 24]))
HP_DENSE_UNITS = hp.HParam('dense_units', hp.Discrete([100, 200, 400]))


In [ ]:
def train_model(hparams):

  model = keras.Sequential([
    keras.layers.Conv2D(kernel_size = 3, filters = hparams[HP_CONV_UNITS] use_bias = False, padding = 'same', input_shape = (28, 28, 1)),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('gelu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 24, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Flatten(),

    keras.layers.Dense(hparams[HP_DENSE_UNITS], use_bias = False),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

  model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
  _, accuracy = model.evaluate(test_images, test_labels)
  return accuracy


In [ ]:
num_conv_units = []
num_dense_units = []
units_accuracy = []

for conv_units in HP_CONV_UNITS.domain.values:
    for dense_units in HP_DENSE_UNITS.domain.values:
        hparams = { 
            HP_CONV_UNITS: conv_units,
            HP_DENSE_UNITS: dense_units,
        }
        accuracy = train_model(hparams)
        num_conv_units.append(conv_units)
        num_dense_units.append(dense_units)
        units_accuracy.append(accuracy)

In [ ]:
for i in range (9):
        print("---------------------------------------------------------------------------")
        print("CONV Layer Filters: " + str(num_conv_units[i]))
        print("DENSE Layer Units: " + str(num_dense_units[i]))
        print("Accuracy: " + str(units_accuracy[i]))
        print("---------------------------------------------------------------------------")